# File Reading 

- [**Imports**](#Imports)
- [**Introduction**](#Introduction)
- [**Reading EDF Files**](#Reading-EDF-Files)
    - [**EDF Reader**](#EDF-Reader)
    - [**Properties and Attributes**](#Properties-and-Attributes)
    - [**Header Information**](#Header-Information)
        - be sure to include how to access header items using '.' dot notation
    - [**File Resources**](#File-Resources)
        - discuss closing of open file readers
    - [**EDF Context Managers**](#EDF-Context-Managers)
- [**Writing EDF Files**](#Writing-EDF-Files)
    - open the demo file and select only channels 1 and 3 and write them to a new file in demos.
    - make sure the header matches the data in the written file
    - open the newly saved file and check that its data matches the original file for chs 1 and 3 
- [**Building New Reader Types**](#Building-New-Reader-Types)
    - discuss what is required of any reader type 
    - discuss how these requirements satisfy production of subarrays using producers from readers

## Objectives for 7/7 - 7-14

- 1. Review producers demo checking for typos, markdown errors, and hyperlink errors.
- 2. Review differences between our producer demos. A few items I noticed:
    - a. Code simplicity -- my largest cell contains only 8 lines. Demos should use only small code snippets.
    - b. Show call signatures and help documentation
    - c. Highlight important points with text boldings, colors etc.
    - d. print what instances look like on return where needed
    - e. limit use of subfunctions with kwargs set to specific values; delegate kwargs to caller function (65)
    - f. realistic examples -see the sizes of arrays chosen, the selection of items to mask in producers etc.
- 3. This file reading demo